In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Necessary Library**

In [28]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

**Data Loading**

In [10]:
train_data=pd.read_csv('train_data.csv')
train_data.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No


**Data Manupulation**

In [11]:
income_range=['Less than ₹12500','₹12500 - ₹24999','₹25000 - ₹37499','₹37500 - ₹49999','₹50000 - ₹62499','₹62500 - ₹74999',
              '₹75000 - ₹87499','₹75000 - ₹87499','₹87500 - ₹99999','₹100000 or More']
no_visited_Cold_drinks=['never','less1','1~3','4~8','gt8']    
offer_expiration=['twodays','tenhours'] 
Restaur_spend_less_than20=['never','less1','1~3','4~8','gt8']
Restaur_spend_greater_than20=['never','less1','1~3','4~8','gt8']
no_visited_bars=['never','less1','1~3','4~8','gt8']
Qualification=['Some High School','High School Graduate','Some college - no degree','Associates degree','Bachelors degree','Graduate degree (Masters or Doctorate)']
no_Take_aways=['never','less1','1~3','4~8','gt8']

In [12]:
train_data.replace(' ',np.nan,inplace=True)

train_data['offer expiration'].replace('2days','0',inplace=True)
train_data['offer expiration'].replace('10hours','1',inplace=True)

train_data['age'].replace('50plus',50,inplace=True)
train_data['age'].replace('below21',20,inplace=True)

train_data.isnull().sum()

offer expiration                            0
income_range                                0
no_visited_Cold drinks                    198
travelled_more_than_15mins_for_offer        0
Restaur_spend_less_than20                 121
Marital Status                              0
restaurant type                             0
age                                         0
Prefer western over chinese                 0
travelled_more_than_25mins_for_offer        0
travelled_more_than_5mins_for_offer         0
no_visited_bars                            93
gender                                      0
car                                     12268
restuarant_same_direction_house             0
Cooks regularly                             0
Customer type                               0
Qualification                               0
is foodie                                   0
no_Take-aways                             144
Job/Job Industry                            0
restuarant_opposite_direction_hous

In [13]:
train_data.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,0,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,0,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,0,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,1,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,0,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No


**Feature Extraction**

In [14]:
y_train=train_data['Offer Accepted']

x_train=train_data.drop(['car','Offer Accepted'],axis=1)

In [15]:
x_train.isnull().sum()

offer expiration                          0
income_range                              0
no_visited_Cold drinks                  198
travelled_more_than_15mins_for_offer      0
Restaur_spend_less_than20               121
Marital Status                            0
restaurant type                           0
age                                       0
Prefer western over chinese               0
travelled_more_than_25mins_for_offer      0
travelled_more_than_5mins_for_offer       0
no_visited_bars                          93
gender                                    0
restuarant_same_direction_house           0
Cooks regularly                           0
Customer type                             0
Qualification                             0
is foodie                                 0
no_Take-aways                           144
Job/Job Industry                          0
restuarant_opposite_direction_house       0
has Children                              0
visit restaurant with rating (av

**Handling missing values**

In [16]:
si=SimpleImputer(missing_values=np.nan,strategy='most_frequent')

x_trans=si.fit_transform(x_train)
x_trans

array([['0', '₹100000 or More', '4~8', ..., 'Spring', 'Location B', 0],
       ['0', '₹87500 - ₹99999', '4~8', ..., 'Summer', 'Location B', 0],
       ['0', '₹87500 - ₹99999', 'less1', ..., 'Winter', 'Location A', 1],
       ...,
       ['1', '₹87500 - ₹99999', '4~8', ..., 'Spring', 'Location B', 1],
       ['1', '₹12500 - ₹24999', '4~8', ..., 'Summer', 'Location C', 0],
       ['0', '₹37500 - ₹49999', '1~3', ..., 'Spring', 'Location A', 1]],
      dtype=object)

In [17]:
df=pd.DataFrame(x_trans,columns=['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
        'restuarant_same_direction_house', 'Cooks regularly',
       'Customer type', 'Qualification', 'is foodie', 'no_Take-aways',
       'Job/Job Industry', 'restuarant_opposite_direction_house',
       'has Children', 'visit restaurant with rating (avg)', 'temperature',
       'Restaur_spend_greater_than20', 'Travel Time', 'Climate',
       'drop location', 'Prefer home food'])
a=df[['income_range','Restaur_spend_less_than20','Restaur_spend_greater_than20','no_visited_Cold drinks','no_visited_bars','Qualification','no_Take-aways']]
a

,income_range,Restaur_spend_less_than20,Restaur_spend_greater_than20,no_visited_Cold drinks,no_visited_bars,Qualification,no_Take-aways
0,₹100000 or More,less1,less1,4~8,less1,Bachelors degree,1~3
1,₹87500 - ₹99999,4~8,1~3,4~8,never,Bachelors degree,1~3
2,₹87500 - ₹99999,1~3,less1,less1,never,Bachelors degree,1~3
3,₹37500 - ₹49999,1~3,1~3,less1,never,Some college - no degree,1~3
4,₹100000 or More,1~3,less1,never,less1,Some college - no degree,gt8
...,...,...,...,...,...,...,...
12374,₹25000 - ₹37499,gt8,never,less1,never,Some college - no degree,less1
12375,₹25000 - ₹37499,never,never,4~8,1~3,Some college - no degree,less1
12376,₹87500 - ₹99999,gt8,gt8,4~8,1~3,Bachelors degree,1~3
12377,₹12500 - ₹24999,less1,less1,4~8,never,Bachelors degree,4~8


In [18]:
oe=OrdinalEncoder(categories=[income_range,Restaur_spend_less_than20,Restaur_spend_greater_than20,no_visited_Cold_drinks,
                              no_visited_bars,Qualification,no_Take_aways])
oe.fit(a)
b=oe.transform(a)
c=pd.DataFrame(b)
m1_data=df.join(c)
m1_data


,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,Climate,drop location,Prefer home food,0,1,2,3,4,5,6
0,0,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,Spring,Location B,0,9.0,1.0,1.0,3.0,1.0,4.0,2.0
1,0,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50,0,0,...,Summer,Location B,0,8.0,3.0,2.0,3.0,0.0,4.0,2.0
2,0,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,Winter,Location A,1,8.0,2.0,1.0,1.0,0.0,4.0,2.0
3,1,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,Summer,Location C,0,3.0,2.0,2.0,1.0,0.0,2.0,2.0
4,0,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,Summer,Location C,0,9.0,2.0,1.0,0.0,1.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,1,₹25000 - ₹37499,less1,1,gt8,Unmarried partner,Restaurant with pub,50,0,0,...,Summer,Location B,0,2.0,4.0,0.0,1.0,0.0,2.0,1.0
12375,0,₹25000 - ₹37499,4~8,0,never,Single,Take-away restaurant,21,0,0,...,Summer,Location B,0,2.0,0.0,0.0,3.0,2.0,2.0,1.0
12376,1,₹87500 - ₹99999,4~8,1,gt8,Married partner,Cold drinks,31,0,1,...,Spring,Location B,1,8.0,4.0,4.0,3.0,2.0,4.0,2.0
12377,1,₹12500 - ₹24999,4~8,1,less1,Married partner,2 star restaurant,26,0,0,...,Summer,Location C,0,1.0,1.0,1.0,3.0,0.0,4.0,3.0


In [19]:
le=LabelEncoder()
y_tra_train=le.fit_transform(y_train)
y_tra_train

array([0, 1, 1, ..., 0, 1, 1])

In [20]:
ohe=OneHotEncoder()
feature_array=ohe.fit_transform(m1_data[['Marital Status','gender','restaurant type','Customer type','Climate','drop location']]).toarray()
columns=np.concatenate(ohe.categories_)
feature_pd=pd.DataFrame(feature_array,columns=columns)
m1_data=m1_data.reindex(columns=m1_data.columns)
m2_data=m1_data[['offer expiration','travelled_more_than_15mins_for_offer','age','Prefer western over chinese','travelled_more_than_5mins_for_offer',
                 'restuarant_same_direction_house','Cooks regularly','is foodie','Job/Job Industry', 'restuarant_opposite_direction_house',
                 'has Children', 'visit restaurant with rating (avg)', 'temperature','Travel Time', 'Prefer home food',0,1,2,3,4,5,6]]
m2_data=m2_data.join(feature_pd)
m2_data.rename(columns={0:'income_range',1:'Restaur_spend_less_than20',2:'Restaur_spend_greater_than20',3:'no_visited_Cold_drinks',
                        4:'no_visited_bars',5:'Qualification',6:'no_Take_aways'})
m2_data['Job/Job Industry']=le.fit_transform(m2_data['Job/Job Industry'])
m2_data['Job/Job Industry']
m2_data



,offer expiration,travelled_more_than_15mins_for_offer,age,Prefer western over chinese,travelled_more_than_5mins_for_offer,restuarant_same_direction_house,Cooks regularly,is foodie,Job/Job Industry,restuarant_opposite_direction_house,...,Individual,With Colleagues,With Family,With Kids,Spring,Summer,Winter,Location A,Location B,Location C
0,0,1,36,0,1,0,1,0,24,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0,0,50,0,1,1,1,0,24,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0,1,26,1,1,1,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1,0,46,1,1,1,0,1,21,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,1,21,0,1,0,0,1,21,1,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,1,1,50,0,1,0,0,1,21,1,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
12375,0,0,21,0,1,0,0,1,17,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
12376,1,1,31,0,1,0,1,1,15,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12377,1,1,26,0,1,0,1,0,22,1,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(m2_data,y_tra_train,test_size=0.2,random_state=42,shuffle=True)

In [23]:
X_test

,offer expiration,travelled_more_than_15mins_for_offer,age,Prefer western over chinese,travelled_more_than_5mins_for_offer,restuarant_same_direction_house,Cooks regularly,is foodie,Job/Job Industry,restuarant_opposite_direction_house,...,Individual,With Colleagues,With Family,With Kids,Spring,Summer,Winter,Location A,Location B,Location C
2442,0,1,46,0,1,1,0,0,10,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
970,1,1,21,1,1,0,0,0,22,1,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5763,0,0,41,0,1,0,1,1,5,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6621,1,0,31,0,1,1,0,0,3,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
11410,1,1,50,1,1,0,0,0,17,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7058,0,1,26,1,1,0,1,1,11,1,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
10254,0,1,31,1,1,0,0,0,5,1,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
12202,1,1,36,1,1,0,1,0,2,1,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2619,1,1,26,1,1,0,0,1,0,1,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [36]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 42)

In [37]:
model.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [40]:
from sklearn.metrics import precision_recall_fscore_support as score

print("score:",model.score(X_test,y_test))

y_pred_test = model.predict(X_test) # predictions on validation test data

precision,recall,fscore,support = score(y_test,y_pred_test,average = "macro")
print('F-score : {},'.format(fscore))

score: 0.630048465266559
F-score : 0.6024840714317763,


In [ ]:
predictions = model.predict(kaggle_test_data)  # predictions on the test dataset
submission = pd.DataFrame(columns=["id", "Offer Accepted"])  # create an empty dataframe with two columns namely "id" and "offer Accepted"
submission["id"] = [i for i in range(len(predictions))]  # id is nothing just serial number
submission["Offer Accepted"] = predictions

submission.to_csv("submission.csv",index = False)